# 1-Generate Predictions

- **Goal:** Prediction Recognition

- **Purpose:** To implement step 1 with sub steps of prediction recognition pipeline. See steps
    1. Generate predictions
        1. Create several prediction prompts templates
        2. Utilize open-source LLMs to generate predictions

- **Misc:**
    - `%store`: Cell magic will store the variable of interest so we can load in another notebook

In [1]:
import os
import sys

import pandas as pd

from pathlib import Path

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from pipelines import BasePipeline

In [2]:
prediction_template = """My variables are
- $\hat{y}$, prediction
    - $\hat{y}_{s}$, source that predicted $\hat{y}$
        - Source can be person, organization, and any type of entity.
        - Analyst forecasts, financial reports, company executives, etc.
        - Meteorologists, weather organizations, or any type of weather-predicting entity.
        - Health organization, researcher, doctor, physical therapist, physician assistant, nurse practictioners, fitness expert, etc.
    - $\hat{y}_{t}$, time when $\hat{y}$ was made
        - Time is the exact moment that can be measured in day, hour, minutes, seconds, etc.
    - $\hat{y}_{f}$, forecast time when $\hat{y}$ is expected to come to fruition
        - Forecast can be from seconds to decades in the future.
        - How far to go out? Or where to stop?
    - $\hat{y}_{a}$, prediction attribute
        - Financial based attributes such as stock price, net profit, revenue, etc.
        - Weather-based attributes such as temperature, precipitation, wind speed, humidity, etc.
        - Health-based attributes such as obesity rates, prevalence of chronic illnesses, average physical activity levels, nutritional intake, etc.
    - $\hat{y}_{m}$, prediction metric outcome
        - How much will the $\hat{y}_{a}$ rise/increase, fall/decrease, change, stay stable
    - $\hat{y}_{v}$, future verb tense
        - A verb that is associated with the future such as will, would, be going to, should, etc.

- Requirements to use for each prediction
    1. Should be based on real-world earnings reports
    2. Only a simple sentence (prediction) (and NOT compounding using "and" or "or")
    3. Should be either positive, negative, or neutral for metric outcome
    4. Suppose the time when $\hat{y}$ was made is during any earning season
    5. Include attributes ($\hat{y}_{a}$) like stock price, net profit, revenue, temperature, precipitation, wind speed, etc.
    6. Include at least 5 stocks from all industries such as technology, energy, etc
    7. Should diversity the metric outcome (e.g., temperature rising 5 degrees, rainfall increasing by 20%).
    8. Should use any future tense word such as will, may, should, could, etc and phrases such as high chance/probability/degree of...
    9. Should have a forecast time when $\hat{y}$ is expected to come to fruition ($\hat{y}_{f}$) between 2025 to 2030
    10. Diversity the name ($\hat{y}_{s}$)
    11. Should use synonyms of predicts such as forecasts, speculates, forsee, envision, etc
    12. Only include the predictions without "Here are 10 company-based financial prediction..." or anything similar and without the numbers in front
    13. Use the five different templates above
    14. The prediction should be unique and not repeated
    15. The $\hat{y}_{f}$ should always be after $\hat{y}_{t}$
    16. Do not number the preditions
    17. Do not say, "As the Chief Financial Officer at a publicly traded company on the US Stock Exchange, I will generate five company-based financial predictions using the provided templates." or "As the Chief Meteorologist at a national weather forecasting agency, I will generate five weather-based predictions using the provided templates.", "As the Chief Financial Officer at a publicly traded company on the US Stock Exchange, I will generate five company-based financial predictions using the provided templates.", or anything similar

- Financial Template 1: On [ $\hat{y}_{t}$,], [$\hat{y}_{s}$ person name] predicts that the [ $\hat{y}_{a}$] at [ $\hat{y}_{s}$ company name] [ $\hat{y}_{v}$] [ $\hat{y}_{m}$] by [$\hat{y}_{m}$, ] in [ $\hat{y}_{f}$]
- Template 2: On [ $\hat{y}_{t}$ ], [ $\hat{y}_{s}$ person name ] from [ $\hat{y}_{s}$ company name ] predicts that the [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] by [ $\hat{y}_{m}$ ] in [ $\hat{y}_{f}$ ].
- Template 3: [ $\hat{y}_{s}$ person name ] predicts on [ $\hat{y}_{t}$ ] that the [ $\hat{y}_{a}$ ] at [ $\hat{y}_{s}$ company name ] [ $\hat{y}_{v}$ ] by [ $\hat{y}_{m}$ ] in [ $\hat{y}_{f}$ ].
- Template 4: According to [ $\hat{y}_{s}$ person name ] from [ $\hat{y}_{s}$ company name ], on [ $\hat{y}_{t}$ ], the [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] by [ $\hat{y}_{m}$ ] in the timeframe of [ $\hat{y}_{f}$ ].
- Template 5: In [ $\hat{y}_{f}$ ], the [ $\hat{y}_a$ ] at [ $\hat{y}_s$ company name ] is expected to [ $\hat{y}_v$ ] by [ $\hat{y}_m$ ], as predicted by [ $\hat{y}s$ person name ] on [ $\hat{y}{t}$ ].

Suppose you are the Chief Financial Officer at a publicly traded company on the US Stock Exchange. Using the above templates, please generate one of each (so five total) company-based financial prediction that will occur in the future following the requirements above.

- Examples for template 1:
    1. On [Monday, December 16, 2024], [Detravious] forecasts that the [revenue] at [Apple] [will] [rise] from [8% to $120 per share] in [Q1 of 2025].
    2. On [Tuesday, November 19, 2024], [Ava Lee] predicts that the [operating cash flow] at [ExxonMobil (XOM)] [should] [decrease] by [5% to $20 billion] in [Q2 of 2027].
- Examples for template 2:
    3. On [Wednesday, October 23, 2024], [Julian Hall] at [NVIDIA (NVDA)] envisions that the [stock price] [will likely] [rise] from [25% to $1,000 per share] in [Q3 of 2028].
    4. On [Thursday, September 19, 2024], [Raj Taylor] from [McDonald's (MCD)] predicts that the [net profit] [will] [fall] by [5% to $5 billion] in [Q4 of 2026].
- Examples for template 3:
    6. [Ava Lee] predicts on [Wednesday, October 23, 2024], that the [research and development expenses] at [Alphabet (GOOGL)] [will] [rise] by [10%] to [$20 billion] in [Q4 of 2027].
    7. [Michael Johnson] predicts on [Monday, March 18, 2024], that the [operating income] at [Microsoft (MSFT)] [will] [fall] from [15% to $50 billion] in [Q2 of 2026].
- Examples for template 4:
    8. According to [Ava Morales] from [Chevron (CVX)], on [Wednesday, August 21, 2024], the [net profit at Coca-Cola (KO)] [is expected to] [increase] from [5% to $10 billion] in the timeframe of [Q3 of 2029].
    9. According to [Sophia Martinez] from [Tesla (TSLA)], on [Friday, July 12, 2024], the [gross profit] [is expected to] [increase] by [15% to $30 billion] in the timeframe of [Q1 of 2026].
- Examples for template 5:
    10. In [Q1 of 2026], the [net profit] at [Amazon (AMZN)] is expected to [increase] from [10% to $15 billion], as predicted by [Emily Davis] on [Monday, January 15, 2024].
    11. In [Q3 of 2027], the [revenue] at [Facebook (META)] is expected to [rise] by [20% to $50 billion], as predicted by [John Smith] on [Tuesday, February 20, 2024].

Suppose you are the Chief Meteorologist at a national weather forecasting agency. Using the above templates, please generate one of each (so five total) weather-based predictions that will occur in the future following the requirements above.

- Weather Template 1: On [ $\hat{y}_{t}$,], [$\hat{y}_{s}$ meteorologist name] predicts that the [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] [ $\hat{y}_{m}$ ] in [ $\hat{y}_{f}$ ].
- Weather Template 2: On [ $\hat{y}_{t}$ ], [ $\hat{y}_{s}$ meteorologist name ] from [ $\hat{y}_{s}$ weather organization ] forecasts that the [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] by [ $\hat{y}_{m}$ ] in [ $\hat{y}_{f}$ ].
- Weather Template 3: [ $\hat{y}_{s}$ meteorologist name ] predicts on [ $\hat{y}_{t}$ ] that the [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] [ $\hat{y}_{m}$ ] in [ $\hat{y}_{f}$ ].
- Weather Template 4: According to [ $\hat{y}_{s}$ meteorologist name ] from [ $\hat{y}_{s}$ weather organization ], on [ $\hat{y}_{t}$ ], the [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] by [ $\hat{y}_{m}$ ] in the timeframe of [ $\hat{y}_{f}$ ].
- Weather Template 5: In [ $\hat{y}_{f}$ ], the [ $\hat{y}_{a}$ ] is expected to [ $\hat{y}_{v}$ ] by [ $\hat{y}_{m}$ ], as predicted by [ $\hat{y}_{s}$ meteorologist name ] on [ $\hat{y}_{t}$ ].

- Examples for template 1:
    1. On [Tuesday, February 13, 2025], [Dr. Melissa Carter] predicts that the [temperature] [will] [rise] by [5°C] in [New York City] by [Friday, February 16, 2025].
    2. On [Monday, April 8, 2025], [Ethan James] forecasts that [precipitation levels] [are likely to] [increase] by [20%] in [San Francisco] in [May 2025].
- Examples for template 2:
    3. On [Wednesday, March 20, 2025], [Samantha Lin] from [NOAA] forecasts that the [wind speed] [should] [decrease] by [15 mph] in [Chicago] by [Friday, March 22, 2025].
    4. On [Saturday, June 15, 2025], [Carlos Rivera] from [Weather.com] predicts that the [humidity] [will] [rise] by [30%] in [Miami] in [July 2025].
- Examples for template 3:
    5. [Amanda Green] predicts on [Sunday, January 19, 2025] that the [temperature] in [Seattle] [will] [fall] by [10°F] in [late January 2025].
    6. [Tommy Wu] predicts on [Friday, November 22, 2024], that [snowfall levels] in [Denver] [will likely] [increase] by [8 inches] in [December 2024].
- Examples for template 4:
    7. According to [Sophia Lewis] from [AccuWeather], on [Monday, May 6, 2024], the [rainfall] in [Portland] [is expected to] [decrease] by [10%] in the timeframe of [early June 2024].
    8. According to [David Harper] from [Weather Underground], on [Friday, August 9, 2024], the [air quality index] in [Los Angeles] [is likely to] [improve] by [20%] in the timeframe of [fall 2024].
- Examples for template 5:
    9. In [April 2025], the [average temperature] in [Houston] is expected to [rise] by [5°F], as predicted by [Emily Cooper] on [Monday, February 18, 2025].
    10. In [January 2025], the [wind chill] in [Minneapolis] is expected to [fall] by [10°F], as predicted by [James Ortiz] on [Tuesday, December 3, 2024].

Suppose you are a health organization (that the US have), researcher, doctor, fitness expert, etc in the US. Using the above templates, please generate one of each (so five total) health-based predictions that will occur in the future following the requirements above.

- Health Template 1: On [ $\hat{y}_{t}$ ], [$\hat{y}_{s}$ person/organization name] predicts that the [ $\hat{y}_{a}$ ] in [ $\hat{y}_{s}$ location or population ] [ $\hat{y}_{v}$ ] [ $\hat{y}_{m}$ ] below [ $\hat{y}_{f}$ ].
- Health Template 2: On [ $\hat{y}_{t}$ ], [$\hat{y}_{s}$ person/organization name] from [ $\hat{y}_{s}$ institution ] envisions that [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] above [ $\hat{y}_{m}$ ] by [ $\hat{y}_{f}$ ].
- Health Template 3: [$\hat{y}_{s}$ person/organization name] predicts on [ $\hat{y}_{t}$ ] that the [ $\hat{y}_{a}$ ] in [ $\hat{y}_{s}$ location ] [ $\hat{y}_{v}$ ] to [ $\hat{y}_{m}$ ] by [ $\hat{y}_{f}$ ].
- Health Template 4: According to [$\hat{y}_{s}$ person/organization name] from [ $\hat{y}_{s}$ institution ], on [ $\hat{y}_{t}$ ], the [ $\hat{y}_{a}$ ] [ $\hat{y}_{v}$ ] to [ $\hat{y}_{m}$ ] in the timeframe of [ $\hat{y}_{f}$ ].
- Health Template 5: In [ $\hat{y}_{f}$ ], the [ $\hat{y}_{a}$ ] in [ $\hat{y}_{s}$ location ] is [expected to  $\hat{y}_{v}$ ] to [ $\hat{y}_{m}$ ], as predicted by [ $\hat{y}_{s}$ person/organization name ] on [ $\hat{y}_{t}$ ].

- Examples for template 1: 
    1. On [Monday, March 15, 2025], [Dr. Evelyn Carter] predicts that the [obesity rate] in [the United States] [will] [decrease] by [5%] by [2030].
    2. On [Tuesday, June 9, 2025], medical professional [Sophia Rodriguez] predicts that the [cancer rate] in [Georgia] [will] [decrease] by [4 percent] by [2028].	
- Examples for template 2: 
    3. On [Tuesday, June 7, 2025], [Arjun Patel, Ph.D] from [Florida Department of Health] envisions that the [average daily caloric intake] [may] [rise] by [10%] by [2027].
    4. On [Thursday, August 14, 2025], [Dr. Michael Brown] from the [Centers for Disease Control and Prevention] foresee that the [average daily caloric intake] [will] [fall] by [8 percent] in [2027]
- Examples for template 3: 
    5. [The World Health Organization] predicts on [Friday, September 12, 2025] that the [global vaccination rate for measles] [should] [increase] by [15%] by [2028].
    6. [Dr. Sarah Johnson] forecasts on [Monday, October 5, 2025] that the [prevalence of hypertension] in [California] [will] [decrease] by [5 percent] by [2028].
- Examples for template 4: 
    7. According to [Olivia Martinez] from [Stanford University], on [Wednesday, July 10, 2025], the prevalence of [type 2 diabetes in adults] [is expected to decline] by [3%] in the timeframe of [2029].
    8. According to [Rachel Kim, MD] from the [University of California], on [Wednesday, July 16, 2029], the prevalence of [type 2 diabetes in adults] [will] [decline] by [5 percent] in the timeframe of [2030].	
- Examples for template 5: 
    9. In [2026], the [average weekly exercise hours] in [United States] is [expected to rise] by [20%], as predicted by [Harvard School of Public Health] on [Monday, December 1, 2025].
    10. In [2027], the [average weekly walking hours] in the [United States] is [expected to rise] by [15%], as predicted by the [National Institutes of Health] on [Friday, December 12, 2025].	

"""

prediction_label = 1

In [3]:

prediction_label = 1

In [4]:
pd.set_option('max_colwidth', 800)

base_pipeline = BasePipeline()

predictions_df = base_pipeline.generate_predictions(text=prediction_template, label=prediction_label)
predictions_df

,Base Predictions,Prediction Label
0,Here are five company-based financial predictions using the provided templates:,1
1,"On Thursday, November 14, 2024, Emily Chen predicts that the revenue at Amazon will rise by 12% to $150 billion in Q2 of 2026.",1
2,"On Friday, October 18, 2024, David Lee from Microsoft forecasts that the net profit will increase by 8% to $20 billion in Q4 of 2027.",1
3,"Ava Morales predicts on Wednesday, September 11, 2024, that the stock price at Alphabet will fall by 5% to $1,200 per share in Q1 of 2025.",1
4,"According to Sophia Patel from Johnson & Johnson, on Tuesday, August 13, 2024, the operating cash flow is expected to decrease by 3% to $15 billion in the timeframe of Q3 of 2028.",1
5,"In Q2 of 2029, the gross profit at Facebook is expected to increase by 10% to $25 billion, as predicted by Michael Kim on Monday, July 15, 2024.",1
6,Here are five weather-based predictions using the provided templates:,1
7,"On Saturday, March 21, 2025, Dr. Maria Rodriguez predicts that the temperature will rise by 3°C in New York City by Friday, March 28, 2025.",1
8,"On Sunday, April 12, 2025, Ethan Hall from the National Weather Service forecasts that the precipitation levels will decrease by 15% in Los Angeles in May 2025.",1
9,"Samantha Taylor predicts on Monday, February 17, 2025, that the wind speed in Chicago will increase by 10 mph in late February 2025.",1


In [5]:
non_prediction_template = """Generate any sentence that's not a prediction. A prediction is below with variables
    - $\hat{y}$, prediction
        - $\hat{y}_{s}$, source that predicted $\hat{y}$
            - Source can be person, organization, and any type of entity.
        - $\hat{y}_{t}$, time when $\hat{y}$ was made
            - Time is the exact moment that can be measured in day, hour, minutes, seconds, etc.
        - $\hat{y}_{f}$, forecast time when $\hat{y}$ is expected to come to fruition
            - Forecast can be from seconds to decades in the future.
            - How far to go out? Or where to stop?
        - $\hat{y}_a$, prediction attribute
            - Financial based attributes such as stock price, net profit, revenue
        - $\hat{y}_m$, prediction metric outcome
            - How much will the  $\hat{y}_a$ rise/increase or fall/decrease
        - $\hat{y}_v$, future verb tense
            - A verb that is associated with the future such as will, would, be going to, should, etc.

    Please generate nine sentences with the following requirements below:

    1. Only a simple sentence (prediction) (and NOT compounding using "and" or "or")
    2. Include no additional information such as "Here are nine simple sentences that are not predictions:"
"""

non_prediction_label = 0

In [6]:
non_predictions_df = base_pipeline.generate_predictions(text=non_prediction_template, label=non_prediction_label)
non_predictions_df

,Base Predictions,Prediction Label
0,The sun is shining brightly in the clear sky.,0
1,The book has a red cover with golden letters.,0
2,The city is located near the ocean.,0
3,The company has a large number of employees.,0
4,The teacher is writing on the blackboard.,0
5,The students are reading their textbooks quietly.,0
6,The music is playing softly in the background.,0
7,The flowers are blooming in the garden.,0
8,The computer is sitting on the desk.,0


In [7]:
%store predictions_df
%store non_predictions_df

Stored 'predictions_df' (DataFrame)
Stored 'non_predictions_df' (DataFrame)


# References

1. PAPER: [On LLMs-Driven Synthetic Data Generation, Curation, and Evaluation: A Survey](http://arxiv.org/abs/2406.15126)
    - Using this to properly format prompt templates.